In [1]:
import os
import pandas as pd
import numpy as np
import fitsio
import cudf

In [2]:
def extract_coincidentals(spikes_list, idx):
    
    # Spikes coordinates at given wavelength index
    spikes_w = spikes_list[idx]
    # Associated neighbour coordinates
    nb_pixels = index_8nb[spikes_w[0, :], :]
    # Sublist of spikes data that will excludes the one serving as template
    spikes_sublist = spikes_list[:idx]+spikes_list[idx+1:]
    # Coincidental cross-referencing. 
    mask_w_arr = np.array([np.isin(nb_pixels, index_8nb[spikes[0,:], :]).any(axis=1) for spikes in spikes_sublist])
    select_pixels = mask_w_arr.any(axis=0)
    coords_w = spikes_w[0, select_pixels] 
    w_tables = np.insert(mask_w_arr[:, select_pixels], idx, True, axis=0)
    # Retrieve intensity values for the selected coordinates
    intensities = spikes_w[1:, select_pixels]
    arr_w = np.concatenate([coords_w[np.newaxis,...], intensities, w_tables], axis=0)
    
    return arr_w


def cp_insert(a, b, idx, axis):
    c = cp.concatenate([a[:idx,:], b, a[idx:]], axis=axis)
    return c


def gpu_extract_coincidentals(spikes_list, idx):
    
    # Spikes coordinates at given wavelength index
    spikes_w = spikes_list[idx]
    # Associated neighbour coordinates
    nb_pixels = gindex_8nb[spikes_w[0, :], :]
    # Sublist of spikes data that will excludes the one serving as template
    spikes_sublist = spikes_list[:idx]+spikes_list[idx+1:]
    # Coincidental cross-referencing. 
    mask_w_arr = cp.array([cp.isin(nb_pixels, gindex_8nb[spikes[0,:], :]).any(axis=1) for spikes in spikes_sublist])
    select_pixels = mask_w_arr.any(axis=0)
    coords_w = spikes_w[0, select_pixels] 
    w_tables = cp_insert(mask_w_arr[:, select_pixels], cp.ones([1, len(select_pixels)]), idx, 0)
    # Retrieve intensity values for the selected coordinates
    intensities = spikes_w[1:, select_pixels]
    arr_w = cp.concatenate([cp.expand_dim(coords_w, 0), intensities, w_tables], axis=0)
    
    return arr_w
                            

In [3]:
data_dir = os.environ['SPIKESDATA']
spikes_db = pd.read_parquet(os.path.join(data_dir, 'spikes_df_2010.parquet'), engine='pyarrow')

In [4]:
spikes_db2 = spikes_db.set_index(['GroupNumber', 'Time'])

In [5]:
spikes_db2.head()

Path  \
GroupNumber Time                                                                                  
0           2010-05-13 00:00:02.090000+00:00  2010/05/13/2010-05-13T00:00:02.09Z_0193.spikes...   
            2010-05-13 00:00:03.570000+00:00  2010/05/13/2010-05-13T00:00:03.57Z_0094.spikes...   
            2010-05-13 00:00:05.070000+00:00  2010/05/13/2010-05-13T00:00:05.07Z_0335.spikes...   
            2010-05-13 00:00:06.580000+00:00  2010/05/13/2010-05-13T00:00:06.58Z_0171.spikes...   
            2010-05-13 00:00:08.080000+00:00  2010/05/13/2010-05-13T00:00:08.08Z_0211.spikes...   

                                                Size  Wavelength  
GroupNumber Time                                                  
0           2010-05-13 00:00:02.090000+00:00  106560         193  
            2010-05-13 00:00:03.570000+00:00  103680          94  
            2010-05-13 00:00:05.070000+00:00  126720         335  
            2010-05-13 00:00:06.580000+00:00   40320         171  
            2010-05-13 00:00:08.080000+00:00   60480         211

### Get the filepaths (typically 7) for a given group

In [6]:
################################################################################################
# Pre-compute the 8-connectivity lookup table. This will be shared across parallel workers.
################################################################################################
# List of relative 2D coordinates for 8-neighbour connectiviy (9-element list). 1st one is the origin pixel.
coords_8nb = np.array([[0, 0], [-1, 0], [-1, -1], [0, -1], [1, -1], [1, 0], [1, 1], [0, 1], [-1, 1]])
# Array of 2D coordinates for a 4096 x 4096 array. Matrix convention is kept. [rows, cols] = [y-axis, x-axis]
ny, nx = [4096, 4096]
coords_1d = np.arange(nx * ny)
coordy, coordx = np.unravel_index(coords_1d, [ny, nx]) # also possible by raveling a meshgrid() output
coords2d = np.array([coordy, coordx])
# Create the array of 2D coordinates of 8-neighbours associated with each pixel.
# pixel 0 has 8 neighbour + itself, pixel 1 has 8 neighbour + itself, etc...
coords2d_8nb = coords2d[np.newaxis, ...] + coords_8nb[..., np.newaxis]
# Handle off-edges coordinates by clipping to the edges, operation done in-place. Here, square detector assumed. Update
# to per-axis clipping if that ever changes for another instrument.
np.clip(coords2d_8nb, 0, nx-1, out=coords2d_8nb)
# Convert to 1D coordinates.
index_8nb = np.array([coords2d_8nb[i, 0, :] * nx + coords2d_8nb[i, 1, :] for i in range(len(coords_8nb))],
                     dtype='int32', order='C').T
index_8nb.shape

(16777216, 9)

In [9]:
n_co_spikes = 2

group_n = 0
fpaths = spikes_db2.loc[group_n]['Path'].values
spikes_list = [fitsio.read(os.path.join(data_dir, f)) for f in fpaths]
for spikes in spikes_list:
    print(spikes.shape)

(3, 8486)
(3, 30356)
(3, 36549)
(3, 7993)
(3, 13781)
(3, 26443)
(3, 27576)


In [ ]:
column_names = ['coords' , 'int1', 'int2', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7']
#column_names_list = [[names for names in column_names[:i]+column_names[i+1:]] for i in range(7)]

df = pd.DataFrame(columns=column_names)
df.head()

In [ ]:
spikes_pix = [[spikes[0,:] for spikes in spikes_list[:i]+spikes_list[i+1:]] for i in range(7)]
pixels_ws = [spikes_list[i][0,:] for i in range(7)]

In [ ]:
%%timeit
group_data = np.concatenate([extract_coincidentals(spikes_list, i) for i in range(7)], axis=1)
u, idx = np.unique(group_data[0, :], return_index=True)
group_data2 = group_data[:, idx]
df = pd.DataFrame(group_data2.T, columns=column_names)

In [ ]:
idx = 0
# Spikes coordinates at given wavelength index
spikes_w = spikes_list[idx]
# Associated neighbour coordinates
nb_pixels = index_8nb[spikes_w[0, :], :]
# Sublist of spikes data that will excludes the one serving as template
spikes_sublist = spikes_list[:idx]+spikes_list[idx+1:]
# Coincidental cross-referencing. 
a = index_8nb[spikes_sublist[0][0,:], :]
#mask_w_arr = np.array([np.isin(nb_pixels, index_8nb[spikes[0,:], :]).any(axis=1) for spikes in spikes_sublist])
m = np.isin(nb_pixels, index_8nb[spikes_sublist[0][0,:], :])
print(nb_pixels.shape)
print(index_8nb[spikes_sublist[0][0,:], :].shape)

In [ ]:
%timeit nb_pixels = index_8nb[spikes_w[0, :], :]

## Test GPU version

In [ ]:
import cudf

In [ ]:
print(index_8nb.dtype)
tempdf = pd.DataFrame(index_8nb)
tempdf.head()
print(index_8nb.nbytes/(1024**2))

In [ ]:
gdf8nb = cudf.from_pandas(tempdf)
gdf8nb.head()

In [ ]:
idx = 0
# Spikes coordinates at given wavelength index
coords = spikes_list[idx][0]
s1 = cudf.Series(coords)
print(s1.head())

# Associated neighbour coordinates
# nb_pixels = gindex_8nb[spikes_w[0, :], :]
# # Sublist of spikes data that will excludes the one serving as template
# spikes_sublist = spikes_glist[:idx]+spikes_glist[idx+1:]
# print(nb_pixels.shape)

In [ ]:
%timeit a = gdf8nb.loc[coords]

In [ ]:
b = gdf8nb.loc[s1]

In [ ]:
del gdf8nb

In [ ]:
# Coincidental cross-referencing. 
# mask_w_arr = cp.array([cp.isin(nb_pixels, gindex_8nb[spikes[0,:], :]).any(axis=1) for spikes in spikes_sublist])

a = np.random.randint(1, high=4096*4096, size=750000)
b = np.random.randint(1, high=4096*4096, size=273204)
c = np.isin(a, b)

g1 = cp.random.randint(1, high=4096*4096, size=75000)
g2 = cp.random.randint(1, high=4096*4096, size=300000)
g3 = cp.in1d(g1, g2)


In [ ]:

select_pixels = mask_w_arr.any(axis=0)
coords_w = spikes_w[0, select_pixels] 
w_tables = cp_insert(mask_w_arr[:, select_pixels], cp.ones([1, len(select_pixels)]), idx, 0)
# Retrieve intensity values for the selected coordinates
intensities = spikes_w[1:, select_pixels]
arr_w = cp.concatenate([cp.expand_dim(coords_w, 0), intensities, w_tables], axis=0)


In [ ]:
ga = cp.arange(10)
ga2 = cp.expand_dims(ga, 0)
print(ga.shape)
print(ga2.shape)

### Test storage format

In [ ]:
mydir = '/home/rattie/Data/AIA_Spikes'

In [ ]:
fitsio.write(mydir+'/data.fits', a_large)
np.savetxt(mydir+'/data.csv', a_large, delimiter=",")

In [ ]:
mydf.to_parquet(mydir+'/data.parquet', engine='pyarrow', compression='None')
mydf.to_parquet(mydir+'/data_compressed_snappy.parquet', engine='pyarrow', compression='snappy')

In [ ]:
fitsf = mydir+'/sample_0193.spikes.fits'
myfits = fitsio.read(fitsf)
myfits.shape

In [ ]:
%timeit fdf = pd.DataFrame(myfits.T, columns=['a', 'b', 'c'])

In [ ]:
parquetf = mydir+'/sample_0193.spikes.compressed_snappy.parquet'
fdf.to_parquet(parquetf, engine='pyarrow', compression='snappy')

In [ ]:
import pyarrow.parquet as pq

In [ ]:
%%timeit
df = pq.read_pandas(parquetf).to_pandas()